In [1]:
import numpy as np
import h5py
from IPython.display import clear_output

import torch

import torchvision.transforms.v2 as transforms

# custom libraries
from utils import *
from celebdf2 import *

import torch.nn as nn
import torch.optim as optim
import torchvision

from torch.utils.data import DataLoader, random_split

from torchvision.models.video import s3d, S3D_Weights
from torchvision.datasets.utils import download_url


### Convert Dataset to HDF5

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transform = transforms.Compose([
    transforms.ConvertImageDtype(torch.float32),    
    transforms.Normalize(mean=(0.43216, 0.394666, 0.37645),
                        std=(0.22803, 0.22145, 0.216989)),
    # transforms.CenterCrop(256),
    transforms.Resize((256,256))
    # ConvertBCHWtoCBHW()
])

dataset_path = "data"
dataset = CelebDF2(dataset_path, transform=transform, max_frames=300, n_frames=150,file_list = 'List_of_testing_videos.txt') # 10s @ 30fps = 300 frames, sample 15 frames per 1s (60,100,150)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_data, val_data = random_split(dataset, [train_size, val_size])
print(f"Training size: {len(train_data)}")
print(f"Validation size: {len(val_data)}")
print(val_data[0][0].shape)
print(val_data[0][1])

Training size: 414
Validation size: 104
torch.Size([3, 150, 256, 256])
1


In [5]:
# Separate images and labels for validation data
val_images = []
val_labels = []

for i in range(len(val_data)):
    # clear_output(wait=False)
    # print(f"val {i}")
    val_images.append(val_data[i][0])  # Assuming images are tensors
    val_labels.append(val_data[i][1])          # Assuming labels are scalar or tensor values

val_images = torch.stack(val_images)  # Stack the images into a single tensor
val_labels = torch.tensor(val_labels)  # Convert labels to tensor if they're not already

# train_images = []
# train_labels = []

# for i in range(len(train_data)):
#     train_images.append(train_data[i][0])  # Assuming images are tensors
#     train_labels.append(train_data[i][1])          # Assuming labels are scalar or tensor values

# train_images = torch.stack(train_images)  # Stack the images into a single tensor
# train_labels = torch.tensor(train_labels)  # Convert labels to tensor if they're not already

# Save the data in a HDF5 file for faster processing and beter RAM utilization
hdf5_path = 'testing_data_val.h5'
with h5py.File(hdf5_path, 'w') as hdf5_file:
    print("image")
    val_images_dataset = hdf5_file.create_dataset(
        'Val_images',
        data=val_images,
        compression="gzip",
        compression_opts=6,
        chunks=True)
    
    print("label")
    val_labels_dataset = hdf5_file.create_dataset(
        'Val_labels',
        data=val_labels,
        compression="gzip",
        compression_opts=6,
        chunks=True)  # Labels should be simpler (e.g., scalar or 1D tensor)
    
    # train_images_dataset = hdf5_file.create_dataset(
    #     'Train_images',
    #     data=train_images,
    #     compression="gzip",
    #     compression_opts=6,
    #     chunks=True)
    
    # train_labels_dataset = hdf5_file.create_dataset(
    #     'Train_labels',
    #     data=train_labels,
    #     compression="gzip",
    #     compression_opts=6,
    #     chunks=True)  # Labels should be simpler (e.g., scalar or 1D tensor)


image
label


In [3]:
# # Save the data in a HDF5 file for faster processing and beter RAM utilization
# hdf5_path = 'testing_data.h5'
# with h5py.File(hdf5_path, 'w') as hdf5_file:
#     # hdf5_file.create_dataset('Train_data', data=train_data, compression="gzip", compression_opts=9)
#     val_images_dataset = hdf5_file.create_dataset(
#         'Val_images',
#         shape=(len(val_data), 3, 150, 256, 256),
#         compression="gzip",
#         compression_opts=6,
#         chunks=True)
    
#     val_labels_dataset = hdf5_file.create_dataset(
#         'Val_labels',
#         shape=(len(val_data),),
#         compression="gzip",
#         compression_opts=6,
#         chunks=True)  # Labels should be simpler (e.g., scalar or 1D tensor)
    
#     train_images_dataset = hdf5_file.create_dataset(
#         'Train_images',
#         shape=(len(train_data), 3, 150, 256, 256),
#         compression="gzip",
#         compression_opts=6,
#         chunks=True)
    
#     train_labels_dataset = hdf5_file.create_dataset(
#         'Train_labels',
#         shape=(len(train_data),),
#         compression="gzip",
#         compression_opts=6,
#         chunks=True)  # Labels should be simpler (e.g., scalar or 1D tensor)

#     for i in range(len(val_data)):
#         # clear_output(wait=False)
#         # print(f"val {i}")
#         val_images_dataset[i] = val_data[i][0] # Assuming images are tensors
#         val_labels_dataset[i] = val_data[i][1]

#     for i in range(len(train_data)):
#         # clear_output(wait=False)
#         # print(f"train {i}")
#         train_images_dataset[i] = train_data[i][0] # Assuming images are tensors
#         train_labels_dataset[i] = train_data[i][1]

NameError: name 'val_data' is not defined

In [2]:
# Define the function to read data from the HDF5 file
def read_hdf5(file_path):
    with h5py.File(file_path, 'r') as hdf5_file:
        # Read the datasets
        val_images = hdf5_file['Val_images'][:]  # Read all data from Val_images
        val_labels = hdf5_file['Val_labels'][:]  # Read all data from Val_labels

        # train_images = hdf5_file['Train_images'][:]  # Read all data from Val_images
        # train_labels = hdf5_file['Train_labels'][:]  # Read all data from Val_labels

    # Convert NumPy arrays to PyTorch tensors if needed
    val_images_tensor = torch.tensor(val_images)
    val_labels_tensor = torch.tensor(val_labels)

    # Convert NumPy arrays to PyTorch tensors if needed
    # train_images_tensor = torch.tensor(train_images)
    # train_labels_tensor = torch.tensor(train_labels)

    return val_images_tensor, val_labels_tensor, #train_images_tensor, train_labels_tensor

# Example usage
hdf5_path = 'testing_data.h5'
val_images_tensor, val_labels_tensor = read_hdf5(hdf5_path) #, train_images_tensor, train_labels_tensor

# Display some information about the loaded data
print(f"Validation Images Shape: {val_images_tensor.shape}")
print(f"Validation Labels Shape: {val_labels_tensor.shape}")

# print(f"Training Images Shape: {train_images_tensor.shape}")
# print(f"Training Labels Shape: {train_labels_tensor.shape}")

Validation Images Shape: torch.Size([104, 3, 150, 256, 256])
Validation Labels Shape: torch.Size([104])


In [ ]:
batch_size = 5
epochs = 15

model = s3d(weights=S3D_Weights.DEFAULT)
freeze(model)
# unfreeze(model)
# replace final layer with new one with appropriate num of classes
model.classifier[1] = nn.Conv3d(1024, 2, kernel_size=1, stride=1)

train_loader = DataLoader(train_data, batch_size, pin_memory=True, num_workers=4, persistent_workers=True) #, num_workers=4, persistent_workers=True
val_loader = DataLoader(val_data, batch_size, pin_memory=True, num_workers=4, persistent_workers=True) # , num_workers=4, persistent_workers=True # maybe just split from training

class_sample_counts = np.array([178, 340])  # Updated with your distribution
class_weights = sum(class_sample_counts) / torch.tensor((class_sample_counts*2), dtype=torch.float)

trainer = BaseTrainer(
        model,
        nn.CrossEntropyLoss(weight=class_weights.to(device)),
        # nn.CrossEntropyLoss(),

        # SGD: overfit/train slow + good generalise well
        optim.SGD(model.parameters(), lr=0.001, momentum=0.9,weight_decay=0.0005), #lr=1e-3,weight_decay=0.0005

        # Adam: overfits/train fast + generalise ok but may not be optimal
        # optim.Adam(model.parameters(), lr=1e-3,weight_decay=0.0005),

        # AdamW: overfits/train very fast + generalise ok
        # optim.AdamW(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.0005), #default: betas=(0.9, 0.999), eps=1e-08, weight_decay=0.01
        
        train_loader,
        val_loader,
        device = device)

train_log, val_log = trainer.fit(epochs)
